In [25]:
import pandas as pd
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [26]:
# Load TSV
df = pd.read_csv("friends_transcripts.tsv", sep="\t")

df.head()


,season_id,episode_id,scene_id,utterance_id,speaker,tokens,transcript
0,s01,e01,c01,u001,Monica Geller,"[['There', ""'s"", 'nothing', 'to', 'tell', '!']...",There's nothing to tell! He's just some guy I ...
1,s01,e01,c01,u002,Joey Tribbiani,"[[""C'mon"", ',', 'you', ""'re"", 'going', 'out', ...","C'mon, you're going out with the guy! There's ..."
2,s01,e01,c01,u003,Chandler Bing,"[['All', 'right', 'Joey', ',', 'be', 'nice', '...","All right Joey, be nice. So does he have a hum..."
3,s01,e01,c01,u004,Phoebe Buffay,"[['Wait', ',', 'does', 'he', 'eat', 'chalk', '...","Wait, does he eat chalk?"
4,s01,e01,c01,u005,unknown,[],NaN


In [27]:
def clean_text(t):
    t = str(t).lower()
    t = re.sub(r"[^a-zA-Z\s]", "", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df["clean_text"] = df["transcript"].apply(clean_text)
df.head()


,season_id,episode_id,scene_id,utterance_id,speaker,tokens,transcript,clean_text
0,s01,e01,c01,u001,Monica Geller,"[['There', ""'s"", 'nothing', 'to', 'tell', '!']...",There's nothing to tell! He's just some guy I ...,theres nothing to tell hes just some guy i wor...
1,s01,e01,c01,u002,Joey Tribbiani,"[[""C'mon"", ',', 'you', ""'re"", 'going', 'out', ...","C'mon, you're going out with the guy! There's ...",cmon youre going out with the guy theres gotta...
2,s01,e01,c01,u003,Chandler Bing,"[['All', 'right', 'Joey', ',', 'be', 'nice', '...","All right Joey, be nice. So does he have a hum...",all right joey be nice so does he have a hump ...
3,s01,e01,c01,u004,Phoebe Buffay,"[['Wait', ',', 'does', 'he', 'eat', 'chalk', '...","Wait, does he eat chalk?",wait does he eat chalk
4,s01,e01,c01,u005,unknown,[],NaN,nan


In [36]:
df = df.sort_values(["season_id", "episode_id", "scene_id", "utterance_id"])

df["prev_text"] = df["clean_text"].shift(1)
df["prev_speaker"] = df["speaker"].shift(1)

pairs = df.dropna(subset=["prev_text"])
pairs = pairs[["prev_text", "clean_text", "speaker"]]
pairs.head()
main_chars = [
    "Ross Geller",
    "Rachel Green",
    "Monica Geller",
    "Phoebe Buffay",
    "Joey Tribbiani",
    "Chandler Bing"
]

pairs = pairs[pairs["speaker"].isin(main_chars)]


In [37]:
# tfidf vectorization
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)

X = vectorizer.fit_transform(pairs["prev_text"])
y = pairs["speaker"]


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [39]:

# Multiclass logistic regression
clf = LogisticRegression(max_iter=3000, n_jobs=-1)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=3000, n_jobs=-1)

In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))


                precision    recall  f1-score   support

 Chandler Bing       0.25      0.22      0.23      1720
Joey Tribbiani       0.28      0.19      0.22      1622
 Monica Geller       0.23      0.18      0.20      1643
 Phoebe Buffay       0.32      0.16      0.21      1580
  Rachel Green       0.25      0.27      0.26      1877
   Ross Geller       0.22      0.43      0.29      1821

      accuracy                           0.25     10263
     macro avg       0.26      0.24      0.24     10263
  weighted avg       0.26      0.25      0.24     10263



In [44]:
def predict_speaker(text):
    x_vec = vectorizer.transform([text])
    return clf.predict(x_vec)[0]

test_line = "We were on a break"
predict_speaker(test_line)


'Ross Geller'